<a href="https://colab.research.google.com/github/rohan-2007/Gemini-Chatbot/blob/main/WhisperAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-xqzhtesb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-xqzhtesb
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

from google.colab import userdata


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest")
genai.configure(api_key = GOOGLE_API_KEY)

In [ ]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  print("start")
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  return 'audio.wav'

record(5)

!whisper audio.wav --model medium.en

input = ""
with open("audio.txt") as f:
  input += f.readline()
  f.close()

print("input: " + input)

response = model.generate_content(input).text
for i in response:
  if not i.isalpha() and not i.isnumeric():
    response.replace(i, "")

print("response: " + response)

from gtts import gTTS

sound = gTTS(text=response, lang="en", slow=False)

sound.save("response.mp3")

from IPython.display import Audio, display

display(Audio("/content/response.mp3", autoplay=True))

<IPython.core.display.Javascript object>

start
[00:00.000 --> 00:03.000]  Give me a bullion about unicorns.
input: Give me a bullion about unicorns.

response: In realms of myth and ancient lore,
Where dreams take flight and legends soar,
A creature pure, a radiant gleam,
The unicorn, a mythical dream.

With spiraling horn, a pearly white,
It strides through forests, bathed in light,
Its coat, a shimmer, silver sheen,
A symbol of grace, a celestial scene.

Its hooves, like diamonds, sparkle bright,
A beacon of hope, a guiding light,
Through misty glades and moonlit trails,
It whispers secrets, never fails.

A creature of magic, strong and bold,
A tale of wonder, yet untold,
The unicorn, a mystic grace,
A timeless beauty, in every space. 



In [ ]:
# audio = record(2)

# with wave.open("audio.wav", "w") as f:
#     # f.setparams((1,2,16000,512,"NONE","NONE"))
#     # f.writeframes(struct.pack("h" * len(audio), *audio))
#     # f.close()
#     f.setnchannels(1)  # Mono audio
#     f.setsampwidth(2)  # 2 bytes per sample
#     f.setframerate(16000)  # 16000 samples per second
#     f.setnframes(512)
#     f.writeframes(struct.pack("h" * len(audio), *audio))

# import sounddevice as sd
# from scipy.io.wavfile import write

# fs = 44100  # Sample rate
# seconds = 3  # Duration of recording

# myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
# sd.wait()  # Wait until recording is finished
# write('output.wav', fs, myrecording)  # Save as WAV file

In [ ]:
# !pip install sounddevice

In [ ]:
!whisper audio.wav --model medium.en

[00:00.000 --> 00:02.000]  Thank you.


In [ ]:
input = ""
with open("audio.txt") as f:
  input += f.readline()
  f.close()

input

'Thank you.\n'

In [ ]:
# !pip install pvrecorder

# from pvrecorder import PvRecorder
# import wave, struct

In [ ]:
# for i, deviceName in enumerate(PvRecorder.get_available_devices()):
#   print(str(i) + ", " + deviceName)

In [ ]:
# device = PvRecorder(device_index=0, frame_length=512)
# print(device)
# audio = []

# try:
#   device.start()
#   print("start")

#   while True:
#     frame = device.read()
#     audio.extend(frame)

# except KeyboardInterrupt:
#   device.stop()
#   print("stopped")

#   with wave.open("audio.wav", "w") as f:
#     # f.setparams((1,2,16000,512,"NONE","NONE"))
#     # f.writeframes(struct.pack("h" * len(audio), *audio))
#     # f.close()
#     f.setnchannels(1)  # Mono audio
#     f.setsampwidth(2)  # 2 bytes per sample
#     f.setframerate(16000)  # 16000 samples per second
#     f.setnframes(512)
#     f.writeframes(struct.pack("h" * len(audio), *audio))
# finally:
#   device.delete()

In [ ]:
# !pip install pyaudio

# import pyaudio

In [ ]:
# import sys

# sys.version

In [ ]:
# text

In [ ]:
# import random
# import json
# import pickle
# import numpy as np

# import nltk
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Activation, Dropout
# from tensorflow.keras.optimizers import SGD

# lemmatizer = WordNetLemmatizer()

# intents = json.loads(open('intents2.json').read())

# words = []
# classes = []
# documents = []
# ignore_letters = ["?", "!", ".", ","]

# for intent in intents["intents"]:
#     for pattern in intent["patterns"]:
#         # print(pattern)
#         word_list = nltk.word_tokenize(pattern, language='english', preserve_line=True)
#         words.extend(word_list)
#         documents.append((word_list, intent['tag']))
#         if intent['tag'] not in classes:
#             classes.append(intent['tag'])

# # print(classes)

# words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
# # print(words)
# words = sorted(set(words))

# classes = sorted(set(classes))

# pickle.dump(words, open("words.pkl", 'wb'))
# pickle.dump(classes, open("classes.pkl", 'wb'))

# training = []
# output_empty = [0] * len(classes)

# for document in documents:
#     bag = []
#     word_patterns = document[0]
#     word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
#     for word in words:
#         bag.append(1) if word in word_patterns else bag.append(0)

#     output_row = list(output_empty)
#     output_row[classes.index(document[1])] = 1
#     training.append([bag, output_row])

# random.shuffle(training)

# for i, example in enumerate(training):
#     bag_length = len(example[0])
#     output_length = len(example[1])
#     print(f"Example {i + 1}: Bag Length - {bag_length}, Output Length - {output_length}")

# training = np.array(training, dtype=object)

# train_x = list(training[:,0])
# print(len(train_x[0]))
# train_y = list(training[:,1])
# print(len(train_y[0]))

# model = Sequential()
# model.add(Dense(128, input_shape=(150,), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(len(train_y[0]), activation="softmax"))

# sgd = SGD(momentum = 0.9)
# # lr = 0.01,, nestrov = True

# model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

# hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# model.save("chatbot_model.h5", hist)
# print("Done")

In [ ]:
# import random
# import json
# import pickle
# import numpy as np

# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer

# from tensorflow.keras.models import load_model

# lemmatizer = WordNetLemmatizer()

# intents = json.loads(open('intents2.json').read())

# words = pickle.load(open('words.pkl', 'rb'))
# classes = pickle.load(open('classes.pkl', 'rb'))
# model = load_model("chatbot_model.h5")

# def clean_up_sentence(sentence):
#     sentence_words = nltk.word_tokenize(sentence)
#     sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
#     return sentence_words

# def bag_of_words(sentence):
#     sentence_words = clean_up_sentence(sentence)
#     bag = [0]*len(words)
#     for w in sentence_words:
#         for i,word in enumerate(words):
#             if word == w:
#                 bag[i] = 1
#     return np.array(bag)

# def predict_class(sentence):
#     bow = bag_of_words(sentence)
#     print(np.array([bow]).shape)
#     res = model.predict(np.array([bow]))[0]
#     ERROR_THRESHOLD = 0.25
#     results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD ]

#     results.sort(key=lambda x: x[1], reverse=True)
#     return_list = []
#     for r in results:
#         return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
#     return return_list

# def get_response(intents_list, intents_json):
#     tag = intents_list[0]["intent"]
#     list_of_intents = intents_json['intents']
#     for i in list_of_intents:
#         if i['tag'] == tag:
#             result = random.choice(i['responses'])
#             break
#     return result

# # print("GO! Bot is running!")

# # while True:
# #     message = input("")
# #     ints = predict_class(message)
# #     res = get_response(ints, intents)
# #     print(res)

# message = input
# print(message)
# ints = predict_class(message)
# res = get_response(ints, intents)
# print(res)

response = model.generate_content(input).text
response = response.replace("*", "")
response

"You're welcome! Is there anything else I can help you with? \n"

In [ ]:
# !pip install gtts

In [ ]:
from gtts import gTTS
# from playsound import playsound
# # import Audio
# # from io import BytesIO

# # import os

# # mp3_fp = BytesIO()

sound = gTTS(text=response, lang="en", slow=False)

# # sound.write_to_fp(mp3_fp)
# # mp3_fp.seek(0)

sound.save("response.mp3")

# # os.system("start response.mp3")

# playsound("/content/response.mp3", True)
# # Audio("/content/response.mp3", autoplay=True)

# import simpleaudio as sa

# # Load the sound file
# wave_obj = sa.WaveObject.from_wave_file("/content/audio.wav")

# # Play the sound file
# play_obj = wave_obj.play()

# # Wait for the sound to finish playing
# play_obj.wait_done()

from IPython.display import Audio, display

display(Audio("/content/response.mp3", autoplay=True))